In [29]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef, RDF
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import copy
import pye57 
import ifcopenshell
import multiprocessing as mp
import time
from pathlib import Path
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [197]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
%autoreload 2

In [6]:
path= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"test",'testfiles')#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
ifcPath= os.path.join(path,'IFC','linked project4.ifc' )
graphPath= os.path.join(path,'myAnalysisFolder','BIM','bimGraph.ttl')

In [34]:
BIMNodes=tl.graph_path_to_nodes(graphPath=graphPath,getResource=True)
print(len(BIMNodes))
ifcBeamNodes=[n for n in BIMNodes if 'IfcBeam' in n.className]
print(len(ifcBeamNodes))
ifcColumnNodes=[n for n in BIMNodes if 'IfcColumn' in n.className]
print(len(ifcColumnNodes))

[Open3D WARNING] Unable to load file d:\Scan-to-BIM repository\geomapi\test\testfiles\myAnalysisFolder\BIM\282_SC_f2_Round:Ø30:882687.obj with ASSIMP
[Open3D WARNING] Unable to load file d:\Scan-to-BIM repository\geomapi\test\testfiles\myAnalysisFolder\BIM\282_SC_f2_Round:Ø30:883780.obj with ASSIMP
[Open3D WARNING] Unable to load file d:\Scan-to-BIM repository\geomapi\test\testfiles\myAnalysisFolder\BIM\282_SC_f2_Round:Ø30:883870.obj with ASSIMP
3528
417
220


In [51]:
geometries=[n.resource for n in BIMNodes ]
print(len(geometries))

3528


In [236]:
mygeometries=geometries[0:10]
pcd,array=gmu.create_identity_point_cloud(mygeometries)
print(len(pcd.points))
print(len(array))
# pcdCropped, percentages= gmu.determine_observable_point_cloud(pcd,indentityArray=array)
# print(len(pcdCropped.points))
# print(percentages)

7288
7288


In [239]:
o3d.visualization.draw_geometries([pcd,identityPointCloud,reference])

In [241]:
geometryboxes=[g.get_oriented_bounding_box() for g in geometries if g != None]
mybox=geometryboxes[0]

indices=gmu.get_box_inliers(sourceBox=gmu.expand_box(mybox), testBoxes=geometryboxes) # boxes collision should be tested
print(indices)
mygeometries=[g for i,g in enumerate( geometries) if i in indices]

[0, 933, 1028, 1250, 3024, 3390]


In [232]:
resolution=0.1
identityPointCloud=o3d.geometry.PointCloud()

for geometry in mygeometries:
    referenceGeometries=[g for g in mygeometries if g !=geometry ]
    reference=gmu.join_geometries(referenceGeometries)
    scene = o3d.t.geometry.RaycastingScene()
    cpuReference = o3d.t.geometry.TriangleMesh.from_legacy(reference)
    _ = scene.add_triangles(cpuReference)

    # sample mesh
    area=geometry.get_surface_area()
    count=int(area/(resolution*resolution))
    pcd=geometry.sample_points_uniformly(number_of_points=10*count)
    pcd=pcd.voxel_down_sample(resolution)
    # create identity matrix
    query_points = o3d.core.Tensor(np.asarray(pcd.points), dtype=o3d.core.Dtype.Float32)
    unsigned_distance = scene.compute_distance(query_points)
    occupancy = scene.compute_occupancy(query_points)

    indices=np.where((unsigned_distance.numpy() <=0.5*resolution) & (occupancy.numpy() ==0) )[0]     


    # for i,p in enumerate(pcd.points):
    #     unsigned_distance = scene.compute_distance(p)
    #     occupancy = scene.compute_occupancy(p)

    #     if unsigned_distance <=0.5*resolution and occupancy==0:
    #         indices.append(i)
    
    #add cropped pcd to 
    #report percentage
    pcdCropped = pcd.select_by_index(indices)
    print(len(pcdCropped.points)/len(pcd.points))
    identityPointCloud +=pcdCropped


0.0
0.45353159851301117
0.4365079365079365
0.4777777777777778
0.6444444444444445
0.5624275222265172
0.4773218142548596
0.466955579631636
0.5099337748344371
0.5208791208791209


In [ ]:

    # sources=ut.item_to_list(sources)
    # inliers=[None]*len(sources)

    # test=str(type(reference))
    # #validate reference
    # if ('TriangleMesh' in str(type(reference)) and 
    #     len(reference.triangles)>=2 and
    #     reference.is_watertight()):    
    #     pass
    # elif 'TriangleMesh' in str(type(reference)) and len(reference.triangles)>=2:
    #     reference,_=reference.compute_convex_hull()
    # elif ('PointCloud' in str(type(reference)) and len(reference.points)>=3):
    #     reference,_=reference.compute_convex_hull()
    # else: 
    #     raise ValueError(' Only TriangleMesh and PointCloud allowed')
    #add reference to scene
scene = o3d.t.geometry.RaycastingScene()
reference = o3d.t.geometry.TriangleMesh.from_legacy(reference)
_ = scene.add_triangles(reference)

unsigned_distance = scene.compute_distance(query_point)

for i,s in enumerate(sources):
    #validate sources
    if ('TriangleMesh' in str(type(s)) and 
        len(s.triangles)>=2 and
        s.is_watertight()): 
        pass
    elif 'TriangleMesh' in str(type(s)) and len(s.triangles)>=2:
        s,_=sources[i].compute_convex_hull()
    elif ('PointCloud' in str(type(reference)) and len(reference.points)>=3):
        s,_=sources[i].compute_convex_hull()
    else:
        continue
    
    #sample geometries
    pcd=s.sample_points_uniformly(10)
    query_points = o3d.core.Tensor(np.asarray(pcd.points), dtype=o3d.core.Dtype.Float32)
    occupancy = scene.compute_occupancy(query_points)
    if np.any(occupancy.numpy()):
        inliers[i]=True
        continue
    inliers[i]=False

#select indices    
ind=np.where(np.asarray(inliers) ==True)[0]     
return ind

In [209]:
pointcloud=pcd
resolution=0.1
indentityArray=array

# dists = np.asarray(pointcloud.compute_point_cloud_distance(pointcloud))
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

unique0, counts0 = np.unique(indentityArray, return_counts=True)
indices=[]

for p in pointcloud.points:
    # [_, idx, _] = pcd_tree.search_hybrid_vector_3d(p, radius=0.5*resolution, max_nn=2)
    p=np.asarray(p)
    [_, idx, distance] = pcd_tree.search_radius_vector_xd(p, radius =0.2*resolution)
    
    distance=np.asarray(distance)
    if len(np.asarray(idx))==1:
        indices.append(np.asarray(idx)[0])
    # if len(idx) >1:
    #     indices.append(idx)
pcdCropped = pointcloud.select_by_index(indices)

# print(dists)
# indices = np.where((dists < resolution) & (dists >0.001))[0]
print(len(indices))

1203005


for every object, crop the point cloud around it.

In [116]:
n=1
myNodes=BIMNodes[1]
myNodes
for node in myNodes:
    box=node.get_oriented_bounding_box()
    box=gmu.expand_box(box,u=0.1,v=0.1,w=0.1)
    pcdCropped=pcd.crop(box)
o3d.visualization.draw_geometries([node.resource,myPcd])

TypeError: 'BIMNode' object is not iterable

In [111]:
dists = np.asarray(pcd_raw.compute_point_cloud_distance(pcdCropped))
indices = np.where(dists > 0.00001)[0]
pcd_cropped_inv = pcd_raw.select_by_index(indices)

In [33]:
bimGeometries=[n.resource for n in BIMNodes if ('IfcBeam' not in n.className and 'IfcColumn' not in n.className)]

NameError: name 'BIMNodes' is not defined

In [ ]:
percentages=gmu.determine_percentage_of_coverage(sources: o3d.geometry, reference:o3d.geometry,threshold:float=0.1)